Attribute Information:

The data set contains raw process sensor data (i.e. without feature extraction) which are structured as matrices (tab-delimited) with the rows representing the cycles and the columns the data points within a cycle. The sensors involved are:

Sensor		Physical quantity		Unit		Sampling rate
PS1		Pressure			bar		100 Hz
PS2		Pressure			bar		100 Hz
PS3		Pressure			bar		100 Hz
PS4		Pressure			bar		100 Hz
PS5		Pressure			bar		100 Hz
PS6		Pressure			bar		100 Hz
EPS1		Motor power			W		100 Hz
FS1		Volume flow			l/min		10 Hz
FS2		Volume flow			l/min		10 Hz
TS1		Temperature			Â°C		1 Hz
TS2		Temperature			Â°C		1 Hz
TS3		Temperature			Â°C		1 Hz
TS4		Temperature			Â°C		1 Hz
VS1		Vibration			mm/s		1 Hz
CE		Cooling efficiency (virtual)	%		1 Hz
CP		Cooling power (virtual)		kW		1 Hz
SE		Efficiency factor		%		1 Hz



Has Missing Values?

No

In [68]:
import pandas as pd
import numpy as np
from numpy import loadtxt
import matplotlib.pyplot as plt

#PS: Pressure, bar, 100 Hz --> 100 Messungen pro Skeunde
#EPS: Motor power, W, 100 Hz
#FS: Volume flow, l/min, 10 Hz --> 10 Messungen pro Sekunde
#TS: Temperature, Celsius, 1 Hz --> 1 Messung pro Sekunde
#VS: Vibration, mm/s, 1Hz
#CE: Cooling efficiency (virtual), %, 1 Hz
#CP: Cooling power (virtual), kW, 1 Hz
#SE: Efficency factor, %, 1 Hz


input_path = "input_data/"
file_names = [
    "ce", "cp", "eps1", "se", "vs1", 
    "fs1", "fs2", 
    "ps1", "ps2", "ps3", "ps4", "ps5", "ps6",
    "ts1", "ts2", "ts3", "ts4", "target"
]

data = {}

for file_name in file_names:
    data[file_name] = loadtxt(f"{input_path}{file_name}.txt")

In [ ]:
for file_name in file_names:
    print(f"Shape of {file_name}:{data[file_name].shape}")

In [69]:
df_target = pd.DataFrame(data['target'], columns=['Cooler_Condition',
                                         'Valve_Condition',
                                         'Internal_Pump_Leakage',
                                         'Hydraulic_Accumulator',
                                         'Stable_Flag'])
df_target = df_target['Valve_Condition']

#df_target.head()

In [ ]:
df_target.describe()

In [71]:
del data['target']

In [ ]:
fig, axes = plt.subplots(6, 3, figsize=(15, 20))
axes = axes.flatten()  


for i, key in enumerate(data):
    ax = axes[i]
    ax.plot(data[key], color = 'blue', linewidth = 1)
    ax.plot(np.mean(data[key], axis=1), color = 'red') 
    ax.set_title(key) 

plt.tight_layout()
plt.show()

In [ ]:
for key, array in data.items():
    print(f"{key}: Mean = {round((np.mean(array)),4)}, Std = {round((np.std(array)),4)}, Min = {np.min(array)}, Max = {np.max(array)}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

for key in data: 
    # Erstelle ein 2D-Array (zum Beispiel ein einfaches Bild oder eine Matrix)
    image = data[key]

    # Berechne die 2D-Fourier-Transformation
    f_transform = np.fft.fft2(image)

    # Verschiebe den Nullfrequenzanteil in die Mitte des Spektrums
    f_transform_shifted = np.fft.fftshift(f_transform)

    # Berechne das Magnitudespektrum (Betrag)
    magnitude_spectrum = np.abs(f_transform_shifted)

    # Berechne das Phasenspektrum (Phase)
    phase_spectrum = np.angle(f_transform_shifted)

    # Summiere das Magnitudespektrum entlang der Spalten (horizontale Richtung)
    magnitude_spectrum_sum_cols = np.sum(magnitude_spectrum, axis=0)

    # Plot des Magnitudespektrums als Summe entlang der Spalten
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(magnitude_spectrum_sum_cols)
    plt.title(f"{key}: Magnitude Spectrum (Summe entlang der Spalten)")
    plt.xlabel("Spalten-Index")
    plt.ylabel("Summierte Magnitude")

    # Plot des Phasenspektrums
    plt.subplot(1, 2, 2)
    plt.imshow(phase_spectrum, cmap='twilight', aspect='auto')
    plt.title(f"{key}: Phase Spectrum")
    plt.colorbar(label='Phase (radians)')
    plt.tight_layout()
    plt.show()


In [72]:
import glob
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis
from scipy.signal import find_peaks

import glob
import pandas as pd
import numpy as np

# Funktion zur Feature-Extraktion
def extract_features(data):
    features = {}
    features['mean'] = np.mean(data, axis=0)  # Mittelwert über Spalten
    features['median'] = np.median(data, axis=0)  # Median über Spalten
    features['std'] = np.std(data, axis=0)  # Standardabweichung über Spalten
    features['min'] = np.min(data, axis=0)  # Minimum über Spalten
    features['max'] = np.max(data, axis=0)  # Maximum über Spalten
    features['range'] = np.max(data, axis=0) - np.min(data, axis=0)  # Spannweite
    diffs = np.diff(data, axis=0)  # Differenz entlang der Zeilen
    features['mean_diff'] = np.mean(diffs, axis=0)  # Mittelwert der Differenzen
    features['std_diff'] = np.std(diffs, axis=0)  # Standardabweichung der Differenzen
    return features

for key in data:
    features = extract_features(data[key])
    all_features.append(features)

# In DataFrame speichern
features_df = pd.DataFrame(all_features)


# Speichern
#features_df.to_csv("extracted_features.csv", index=False)


In [ ]:
print(features_df.dtypes)

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

# Extrahierte Features und Labels (ersetze dies mit deinem Datensatz)

# Features und Zielvariable
X = features_df
y = df_target

X.shape

y.shape

# Datenaufteilung
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# AdaBoost-Modell
ada_model = AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=1), n_estimators=50, random_state=42)

# Training
ada_model.fit(X_train, y_train)

# Vorhersagen
y_pred = ada_model.predict(X_test)

# Modellbewertung
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
